In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import pytorch

from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from datasets import load_dataset



ModuleNotFoundError: No module named 'pytorch'

In [39]:
# Load your data
data_dir = 'C:\\Users\\franc\\Documents\\GitHub\\Car_Detection_Chat\\images'

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)


Found 15668 files belonging to 26 classes.
Using 12535 files for training.
Found 15668 files belonging to 26 classes.
Using 3133 files for validation.


In [ ]:
Cars = list(data_dir.glob('roses/*'))
PIL.Image.open(str(roses[0]))


In [7]:
import os
from PIL import Image

for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    img.verify()
            except Exception as e:
                print(f"Problem with file: {file_path}. Error: {e}")


In [10]:
# CHECK ENCODING
def check_utf8_encoding(data_dir):
    for root, dirs, files in os.walk(data_dir):
        # # Check directory names
        # for dir_name in dirs:
        #     try:
        #         dir_name.decode('utf-8')
        #     except UnicodeDecodeError:
        #         print(f"Non-UTF-8 directory name detected: {os.path.join(root, dir_name)}")
        
        # Check file names
        for file_name in files:
            try:
                file_name.encode('utf-8').decode('utf-8')
            except UnicodeDecodeError:
                print(f"Non-UTF-8 file name detected: {os.path.join(root, file_name)}")

data_dir = 'C:\\Users\\franc\\Documents\\GitHub\\Car_Detection_Chat\\images_backup'
check_utf8_encoding(data_dir)